# **Satellite-Image Classification**

## Smit Doshi - 1915123

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import confusion_matrix, classification_report

In [9]:
train_dr = "../input/satellite-image-classification/data"
CLASS = ['cloudy', 'desert', 'green_area', 'water']

In [10]:
img = cv2.imread("../input/satellite-image-classification/data/desert/desert(10).jpg")

In [11]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
print('Image shape :',img.shape,'\n')
plt.imshow(img)
plt.axis('off')
plt.title('Cloudy image')
plt.show()

In [12]:
IMAGE_SIZE = (256)
BATCH_SIZE=32
CHANNELS=3
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
          train_dr,
          validation_split=.3,
          subset="training",
          label_mode="categorical",
          shuffle = True,
          seed=123,
          image_size=(IMAGE_SIZE,IMAGE_SIZE),
          batch_size=BATCH_SIZE)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
          train_dr,
          validation_split=.3,
          subset="validation",
          label_mode="categorical",
          seed=123,
          image_size=(IMAGE_SIZE,IMAGE_SIZE),
          batch_size=BATCH_SIZE)

In [13]:
model =  tf.keras.Sequential([
tf.keras.Sequential([layers.Rescaling(1./255)]),
layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)),
layers.MaxPool2D(pool_size=(2, 2), padding = 'same'),
layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same', strides=1),
layers.MaxPool2D(pool_size=(2, 2)),
layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same', strides=1),
layers.MaxPool2D(pool_size=(2, 2)),
layers.Flatten(),
layers.Dense(32),
layers.Dense(4,activation='softmax')
])



In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy', 
    metrics=['accuracy'])


In [15]:
history = model.fit(
    train_ds, 
    validation_data = val_ds, 
    epochs = 50)

In [16]:
history_df = pd.DataFrame(history.history)
history_df.loc[:,['loss', 'val_loss']].plot()
history_df.loc[:,['accuracy', 'val_accuracy']].plot()

In [17]:
model.summary()

In [18]:
model.evaluate(val_ds)

In [19]:
model.save('model.h5')